In [1]:
# Import workspace and other libraries

import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix

In [2]:
np.set_printoptions(suppress=True)

In [3]:
# Filepath
script_folder = os.getcwd()
filepath = script_folder + '\census-income-data_all_working-class.csv'
print(filepath + '\n')

C:\Users\Alavi\my-folder\census-income-data_all_working-class.csv



In [4]:
# Convert csv to datafreame
df = pd.read_csv(filepath)
print("converted dataset to dataframe\n")

converted dataset to dataframe



In [5]:
## Start preprocessing: Clean up data and normalize

# Replace missing data
df_clean = df.replace('?', np.nan)
print("preprocessing complete\n")

preprocessing complete



In [6]:
df.columns

Index(['Age', 'Class_Of_Worker', 'Industry_Code', 'Occupation_Code',
       'Education', 'Wage_Per_Hour_x_100', 'Enrolled_In_Edu_Inst_Last_Wk',
       'Marital_Status', 'Major_Industry_Code', 'Major_Occupation_Code',
       'Race', 'Hispanic_Origin', 'Sex', 'Member_Of_A_Labor_Union',
       'Reason_For_Unemployment', 'Full_Or_Part_Time_Employment_Stat',
       'Capital_Gains', 'Capital_Losses', 'Divdends_From_Stocks',
       'Region_Of_Previous_Residence', 'State_Of_Previous_Residence',
       'Detailed_Household_And_Family_Stat',
       'Detailed_Household_Summary_In_Household', 'Instance_Weight',
       'Migration_Code-Change_In_Msa', 'Migration_Code-Change_In_Reg',
       'Migration_Code-Move_Within_Reg', 'Live_In_This_House_1_Year_Ago',
       'Migration_Prev_Res_In_Sunbelt', 'Num_Persons_Worked_For_Employer',
       'Family_Members_Under_18', 'Country_Of_Birth_Father',
       'Country_Of_Birth_Mother', 'Country_Of_Birth_Self', 'Citizenship',
       'Own_Business_Or_Self_Employed',

In [7]:
# Separate labels from measures
y_raw = df_clean.pop('Tax_Filer_Status')
#y_clean = y_raw.values.reshape((y_raw.shape[0],1))
X = df_clean
print("label separation complete\n")

label separation complete



In [8]:
# Drop columns with too many missing values (all migration-related columns)
X.drop(X.columns[[24,25,26,28,39]], axis = 1, inplace = True)
print("dropped all migration-related columns and year of survery\n")

dropped all migration-related columns and year of survery



In [9]:
# Define feature names and label/classes
feature_names = np.array(["Age", "Class Of Worker", "Industry Code", "Occupation Code", "Education", 
						"Wage Per Hour x 100", "Enrolled In Educational Inst.", "Marital Status", "Major Industry Code", 
						"Major Occupation Code", "Race", "Hispanic Origin", "Sex", "Member Of A Labour Union", 
						"Reason For Unemployment", "Full Or Part-Time", "Capital Gains", "Capital Losses", 
						"Divdends From Stocks", "Region Of Previous Residence", 
						"State Of Previous Residence", "Detailed Household Stat", "Detailed Household Summary", 
						"Instance Weight", "Lived In This House 1 Year Ago", "Num Persons Worked For Employer", 
						"Family Members Under 18", "Country Of Birth - Father", "Country Of Birth - Mother", 
						"Country Of Birth - Self", "Citizenship", "Self Employed", 
						"Fill Inc Questionnaire For Veteran's Admin", "Veterans Benefits", "Weeks Worked In Year", "Income_<>_$50000"])
print("Defined feature names and classes\n")

Defined feature names and classes



In [10]:
labels = np.array(['tax filer', 'non-filer']).tolist()
print("classes have been defined\n")

classes have been defined



In [11]:
feature_names.shape

(36,)

In [12]:
# Encode categorical values but preserve nulls
le = preprocessing.LabelEncoder()
X = X.apply(lambda series: pd.Series(le.fit_transform(series[series.notnull()]), index=series[series.notnull()].index))
print("label encoding complete\n")

label encoding complete



In [13]:
Y = le.fit_transform(y_raw)

In [14]:
X_train,X_test,y_train,y_test = train_test_split(X, Y, test_size=0.2)
print("split dataset into training and test sets with an 80-20 partition\n")

split dataset into training and test sets with an 80-20 partition



In [15]:
Y[45687:45700]

array([1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1])

In [16]:
# Flatten label column to fit the SVM classifier requirement (otherwise you get warnings)
y_train_flat = y_train.ravel()
y_test_flat = y_test.ravel()
print("flattened class label array\n")

flattened class label array



In [17]:
# Impute missing values using univariate imputer (multivariate is not supported in this version of Scikit-learn 0.18.1)
print("imputing missing values as nan\n")
imp = Imputer(missing_values=np.nan, strategy='most_frequent')
imp.fit(X_train)
X_train_clean = imp.transform(X_train)
X_test_clean = imp.transform(X_test)

print("imputation complete\n")

imputing missing values as nan



C:\Users\Alavi\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


imputation complete



In [18]:
# Normalize dataset (exclude labels)
#X_norm = normalize(X_clean)

# Standardize dataset
scaler = StandardScaler()  

# Fit only on training data
scaler.fit(X_train)  
x_train = scaler.transform(X_train_clean)

# apply same transformation to test data
x_test = scaler.transform(X_test_clean)  

print("standardized dataset\n")

standardized dataset



In [19]:
# Train a shallow neural network
clf = MLPClassifier(solver='sgd',
                    hidden_layer_sizes=(15,30,15), random_state=1)

model = clf.fit(x_train, y_train_flat)

In [20]:
# Train a shallow neural network
clf = MLPClassifier(solver='sgd',
                    hidden_layer_sizes=(10,15,10), random_state=1)

model_1 = clf.fit(x_train, y_train_flat)

In [21]:
len(x_test)

39561

In [22]:
predictions = model.predict(x_test)

In [23]:
print(confusion_matrix(y_test_flat,predictions))

[[ 5476  1876]
 [ 1109 31100]]


In [24]:
print(classification_report(y_test_flat,predictions))

              precision    recall  f1-score   support

           0       0.83      0.74      0.79      7352
           1       0.94      0.97      0.95     32209

    accuracy                           0.92     39561
   macro avg       0.89      0.86      0.87     39561
weighted avg       0.92      0.92      0.92     39561



In [25]:
y_test_flat[220:231]

array([0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1])

In [26]:
# De-standardize data
x_train_dstd = scaler.inverse_transform(x_train)
x_test_dstd = scaler.inverse_transform(x_test)

In [27]:
X_test['Age'].iloc[0:10].to_numpy()

array([17, 12, 47, 11,  5, 33, 53, 12, 29, 72], dtype=int64)

In [28]:
x_test[0:10,0]

array([-0.60158253, -0.87380648,  1.03176116, -0.92825127, -1.25492001,
        0.2695341 ,  1.3584299 , -0.87380648,  0.05175494,  2.3928809 ])

In [29]:
x_test_dstd[0:10,0].astype(int)

array([17, 12, 47, 11,  5, 33, 53, 12, 29, 72])

In [30]:
from interpret.ext.blackbox import TabularExplainer

explainer = TabularExplainer(model, 
                            x_test_dstd, # x_test
                            features=feature_names, 
                            classes=labels)
print("initialized explainer\n")

Could not import lightgbm, required if using LGBMExplainableModel
Could not import lightgbm, required if using LGBMExplainableModel
Could not import lightgbm, required if using LGBMExplainableModel
Could not import lightgbm, required if using LGBMExplainableModel
Could not import lightgbm, required if using LGBMExplainableModel


C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\linear.py:49: UserWarning: The default value for feature_dependence has been changed to "independent"!
  warnings.warn('The default value for feature_dependence has been changed to "independent"!')


initialized explainer



In [31]:
# you can use the training data or the test data here
global_explanation = explainer.explain_global(x_test_dstd[0:100])

# if you used the PFIExplainer in the previous step, use the next line of code instead
# global_explanation = explainer.explain_global(x_train, true_labels=y_test)

# sorted feature importance values and feature names
sorted_global_importance_values = global_explanation.get_ranked_global_values()
sorted_global_importance_names = global_explanation.get_ranked_global_names()
dict(zip(sorted_global_importance_names, sorted_global_importance_values))

# alternatively, you can print out a dictionary that holds the top K feature names and values
global_explanation.get_feature_importance_dict()

C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C

C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C

C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C

C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C

C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C

C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C

C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C

C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C

{'Income_<>_$50000': 0.0,
 'Weeks Worked In Year': 0.0,
 'Full Or Part-Time': 0.0,
 'Reason For Unemployment': 0.0,
 'Member Of A Labour Union': 0.0,
 'Sex': 0.0,
 'Hispanic Origin': 0.0,
 'Race': 0.0,
 'Major Occupation Code': 0.0,
 'Major Industry Code': 0.0,
 'Marital Status': 0.0,
 'Enrolled In Educational Inst.': 0.0,
 'Wage Per Hour x 100': 0.0,
 'Education': 0.0,
 'Occupation Code': 0.0,
 'Industry Code': 0.0,
 'Class Of Worker': 0.0,
 'Capital Gains': 0.0,
 'Capital Losses': 0.0,
 'Divdends From Stocks': 0.0,
 'Country Of Birth - Father': 0.0,
 'Veterans Benefits': 0.0,
 "Fill Inc Questionnaire For Veteran's Admin": 0.0,
 'Self Employed': 0.0,
 'Citizenship': 0.0,
 'Country Of Birth - Self': 0.0,
 'Country Of Birth - Mother': 0.0,
 'Family Members Under 18': 0.0,
 'Region Of Previous Residence': 0.0,
 'Num Persons Worked For Employer': 0.0,
 'Lived In This House 1 Year Ago': 0.0,
 'Instance Weight': 0.0,
 'Detailed Household Summary': 0.0,
 'Detailed Household Stat': 0.0,
 'Sta

In [32]:
# explain the first data point in the test set
local_explanation = explainer.explain_local(x_test_dstd[0:100])

# sorted feature importance values and feature names
sorted_local_importance_names = local_explanation.get_ranked_local_names()
sorted_local_importance_values = local_explanation.get_ranked_local_values()

C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C

C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C

C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C

C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C

C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C

C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C

C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C

C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C:\Users\Alavi\Anaconda3\lib\site-packages\shap\explainers\kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \
C

In [186]:
# save model

import pickle
model_name = 'xai_poc_model_1.pkl'
pickle.dump(model_1, open(model_name, 'wb'))

In [187]:
# load the model from disk
filename = 'xai_poc_model.pkl'
loaded_model = pickle.load(open(filename, 'rb'))

In [188]:
# load the model from disk
filename1 = 'xai_poc_model_1.pkl'
loaded_model_1 = pickle.load(open(filename1, 'rb'))

In [35]:
from azureml.contrib.interpret.visualize import ExplanationDashboard

ExplanationDashboard(global_explanation, model, x_test_dstd[12345:12445])

ExplanationWidget(value={'predictedY': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…